In [1]:
import numpy as np
import pandas as pd

### Hierarchical Index

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1,2,3,1,3,1,2,2,3]])

In [3]:
data

a  1    0.438798
   2   -0.356302
   3   -0.201225
b  1    1.698636
   3   -0.866713
c  1   -1.716906
   2    0.615640
d  2   -0.168882
   3   -0.412744
dtype: float64

In [4]:
print(data.index)
data.index.levels

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )


FrozenList([['a', 'b', 'c', 'd'], [1, 2, 3]])

In [9]:
data['b':'c']

b  1    0.069819
   3   -0.020976
c  1    0.130671
   2   -0.934089
dtype: float64

In [10]:
data[['b', 'c', 'd']]

b  1    0.069819
   3   -0.020976
c  1    0.130671
   2   -0.934089
d  2    0.074184
   3    0.244452
dtype: float64

In [14]:
data[:, 1] # The position were 1 is written is 2nd layer of index

a   -0.073099
b    0.069819
c    0.130671
dtype: float64

### Stack and Unstack

In [16]:
data.unstack()

,1,2,3
a,-0.073099,0.312066,-0.001579
b,0.069819,NaN,-0.020976
c,0.130671,-0.934089,NaN
d,NaN,0.074184,0.244452


In [17]:
data.unstack().stack()

a  1   -0.073099
   2    0.312066
   3   -0.001579
b  1    0.069819
   3   -0.020976
c  1    0.130671
   2   -0.934089
d  2    0.074184
   3    0.244452
dtype: float64

In [8]:
data.index.names = ['index1', 'index2']

In [9]:
# Unstacking on level 1 indices
data.unstack('index1')

index1,a,b,c,d
index2,,,,
1,0.438798,1.698636,-1.716906,NaN
2,-0.356302,NaN,0.615640,-0.168882
3,-0.201225,-0.866713,NaN,-0.412744


### Hierarchical Columns

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame.index.names = ['Primary_key', 'Secondary_key']
frame.columns.names = ['state', 'color']

frame

state                      Ohio     Colorado
color                     Green Red    Green
Primary_key Secondary_key                   
a           1                 0   1        2
            2                 3   4        5
b           1                 6   7        8
            2                 9  10       11

In [29]:
frame.loc['a', 'Ohio']

color,Green,Red
Secondary_key,,
1,0,1
2,3,4


### Reordering

In [33]:
frame.swaplevel(0, 1)

state                      Ohio     Colorado
color                     Green Red    Green
Secondary_key Primary_key                   
1             a               0   1        2
2             a               3   4        5
1             b               6   7        8
2             b               9  10       11

In [35]:
frame.swaplevel('Secondary_key', 'Primary_key')
# frame.swaplevel('Primary_key', 'Secondary_key') - any way you can swap

state                      Ohio     Colorado
color                     Green Red    Green
Secondary_key Primary_key                   
1             a               0   1        2
2             a               3   4        5
1             b               6   7        8
2             b               9  10       11

### Sorting

In [36]:
# Reording and swaping
frame.swaplevel(0, 1).sort_index(level=1)

state                      Ohio     Colorado
color                     Green Red    Green
Secondary_key Primary_key                   
1             a               0   1        2
2             a               3   4        5
1             b               6   7        8
2             b               9  10       11

In [17]:
frame.sum(axis = 1)

Primary_key  Secondary_key
a            1                 3
             2                12
b            1                21
             2                30
dtype: int64

### setting multiindex from columns

In [47]:
df = pd.DataFrame({
    'a': range(7),
    'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'd': [0,1,2,0,1,2,3]
})

df.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

### Combining and Merging

### Merge()

In [17]:
# Many to One Join

df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': np.arange(7)
})

df2 = pd.DataFrame({
    'key': ['a', 'b', 'd'],
    'data2': np.arange(3)
})

In [18]:
pd.merge(df1, df2) # By Default it is 'inner' join on matching column

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [19]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [20]:
pd.merge(df1, df2, how='outer') # Union merge

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


In [21]:
pd.merge(df1, df2, on='key', how='right')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


In [25]:
# What if the column names are different but the data belongs to same type
df3 = pd.DataFrame({
    'left_key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': np.arange(7)
})

df4 = pd.DataFrame({
    'right_key': ['a', 'b', 'd'],
    'data2': np.arange(3)
})

In [26]:
pd.merge(df3, df4, left_on='left_key', right_on='right_key', how='left')

,left_key,data1,right_key,data2
0,b,0,b,1.0
1,b,1,b,1.0
2,a,2,a,0.0
3,c,3,NaN,NaN
4,a,4,a,0.0
5,a,5,a,0.0
6,b,6,b,1.0


### Hierarchical Indexex - Many to Many join

In [52]:

df5 = pd.DataFrame({
    'key1': ['ohio', 'ohio', 'ohio', 'nevada', 'nevada'],
    'key2': [1,1,2,2,3],
    'data': np.arange(5)
})

df6 = pd.DataFrame({
    'key1': ['nevada', 'nevada', 'ohio', 'ohio', 'ohio', 'ohio'],
    'key2': [2001, 2000, 2000, 2000, 2001, 2002],
    'data': np.arange(6)},
    index=[['a','a','a','b','b','c'], np.arange(6)] # There is no effect of hierarchical indexing on merge
)

pd.merge(df5, df6, on='key1', how='outer')

,key1,key2_x,data_x,key2_y,data_y
0,nevada,2,3,2001,0
1,nevada,2,3,2000,1
2,nevada,3,4,2001,0
3,nevada,3,4,2000,1
4,ohio,1,0,2000,2
5,ohio,1,0,2000,3
6,ohio,1,0,2001,4
7,ohio,1,0,2002,5
8,ohio,1,1,2000,2
9,ohio,1,1,2000,3


### Merging on Index

In [62]:
df7 = pd.DataFrame({
    'key1': ['ohio', 'ohio', 'ohio', 'nevada', 'nevada'],
    'key2': [1,1,2,2,3],
    'data': np.arange(5)
})

df8 = pd.DataFrame(np.arange(12).reshape((6,2)),
    index=[['nevada', 'nevada', 'ohio', 'ohio', 'ohio', 'ohio'], [2,0,1,1,2,2]],
    columns=['feature1', 'feature2']
)

pd.merge(df8, df7, right_on=['key1', 'key2'], left_index=True, how='outer') # In both the cases the multi-index become normal

,feature1,feature2,key1,key2,data
4,2.0,3.0,nevada,0,NaN
3,0.0,1.0,nevada,2,3.0
4,NaN,NaN,nevada,3,4.0
0,4.0,5.0,ohio,1,0.0
1,4.0,5.0,ohio,1,1.0
0,6.0,7.0,ohio,1,0.0
1,6.0,7.0,ohio,1,1.0
2,8.0,9.0,ohio,2,2.0
2,10.0,11.0,ohio,2,2.0


### Join() - Specially for index joining

In [72]:
df5 = pd.DataFrame({
    'key1_1': ['ohio', 'ohio', 'ohio', 'nevada', 'nevada'],
    'key2_1': [1,1,2,2,3],
    'data_1': np.arange(5)
}, index=[['a', 'b', 'c', 'd', 'a'], np.arange(5)])

df6 = pd.DataFrame({
    'key1': ['nevada', 'nevada', 'ohio', 'ohio', 'ohio', 'ohio'],
    'key2': [2001, 2000, 2000, 2000, 2001, 2002],
    'data': np.arange(6)},
    index=[['a','a','a','b','b','c'], np.arange(6)]
    )
df5.join(df6, how='outer')

# Condition - No Overlapping Columns
# Even you can join multiple joins df1.join([df2, df3, df4, ....])

key1_1  key2_1  data_1    key1    key2  data
a 0    ohio     1.0     0.0  nevada  2001.0   0.0
  1     NaN     NaN     NaN  nevada  2000.0   1.0
  2     NaN     NaN     NaN    ohio  2000.0   2.0
  4  nevada     3.0     4.0     NaN     NaN   NaN
b 1    ohio     1.0     1.0     NaN     NaN   NaN
  3     NaN     NaN     NaN    ohio  2000.0   3.0
  4     NaN     NaN     NaN    ohio  2001.0   4.0
c 2    ohio     2.0     2.0     NaN     NaN   NaN
  5     NaN     NaN     NaN    ohio  2002.0   5.0
d 3  nevada     2.0     3.0     NaN     NaN   NaN

In [20]:
students = pd.DataFrame({
    'name': ['Scout', 'Scout', 'Mortal', 'Dynamo', 'Snax', 'Joker'],
    'id': [1,1,2,3,4,5]
})

courses = pd.DataFrame({
    'name':['Scout', 'Scout', 'Scout', 'Mortal', 'Joker'],
    'course_id': [1, 2, 4, 3, 1]
})

pd.merge(students, courses, on='name', how='inner')

,name,id,course_id
0,Scout,1,1
1,Scout,1,2
2,Scout,1,4
3,Scout,1,1
4,Scout,1,2
5,Scout,1,4
6,Mortal,2,3
7,Joker,5,1


### Concatinating along the axis - concat()

In [25]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ['a', 'b', 'c'], columns = ['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index = ['d', 'e'], columns = ['one', 'two'])

pd.concat([df1, df2], keys=['level1', 'level2']) # Hierarchical Indices - axis = 0


one  two
level1 a    0    1
       b    2    3
       c    4    5
level2 d    5    6
       e    7    8

In [26]:
temp = pd.concat([df1, df2], keys=['level1', 'level2'], axis=1, names = ['upper', 'lower']) # Hierarchical columns - axis = 1
temp.index.name = 'Index'

In [27]:
temp

upper level1      level2     
lower    one  two    one  two
Index                        
a        0.0  1.0    NaN  NaN
b        2.0  3.0    NaN  NaN
c        4.0  5.0    NaN  NaN
d        NaN  NaN    5.0  6.0
e        NaN  NaN    7.0  8.0

In [30]:
temp = pd.concat({'df1': df1, 'df2': df2}, axis=1)
temp

df1       df2     
   one  two  one  two
a  0.0  1.0  NaN  NaN
b  2.0  3.0  NaN  NaN
c  4.0  5.0  NaN  NaN
d  NaN  NaN  5.0  6.0
e  NaN  NaN  7.0  8.0